In [1]:
import os
os.environ["GRB_LICENSE_FILE"] = "/Users/yfd/gurobi.lic"

In [2]:
pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import gurobipy
import pyomo.environ as pyo
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import os

In [4]:
# Assume your Excel file is already saved locally
data_file = 'simple_simul_100k.xlsx'

# Make sure the file exists
if not os.path.isfile(data_file):
    raise FileNotFoundError(f"Cannot find {data_file}, please check the file path.")

# Load the Excel file (example)
data = pd.read_excel(data_file)

In [5]:
data.keys()

Index(['Unnamed: 0', 'income_before_tax', 'tax_group', 'household_size',
       'outcome_1', 'outcome_2', 'outcome_3'],
      dtype='object')

In [6]:
data= pd.DataFrame(data)

In [7]:
default_params = {
    "tax_brackets": {
        "all": [
            {"income_up_to": 24812, "rate": 0.3},
            {"income_up_to": 38098, "rate": 0.4},
            {"income_up_to": 75518, "rate": 0.5},
            {"income_up_to": float("inf"), "rate": 0.6},],
        "1": [
            {"income_up_to": 24812, "rate": 0.2},
            {"income_up_to": 38098, "rate": 0.4},
            {"income_up_to": 75518, "rate": 0.4},
            {"income_up_to": float("inf"), "rate": 0.6},],
        "2": [
            {"income_up_to": 24812, "rate": 0.3},
            {"income_up_to": 38098, "rate": 0.3},
            {"income_up_to": 75518, "rate": 0.3},
            {"income_up_to": float("inf"), "rate": 0.5},],
        "3": [
            {"income_up_to": 24812, "rate": 0.1},
            {"income_up_to": 38098, "rate": 0.1},
            {"income_up_to": 75518, "rate": 0.1},
            {"income_up_to": float("inf"), "rate": 0.8},],
        "4": [
            {"income_up_to": 24812, "rate": 0.3},
            {"income_up_to": 38098, "rate": 0.4},
            {"income_up_to": 75518, "rate": 0.1},
            {"income_up_to": float("inf"), "rate": 0.2},],
        "alt_1": [
            {"income_up_to": 20_000, "rate": 0.3},
            {"income_up_to": 50_000, "rate": 0.4},
            {"income_up_to": 100_000, "rate": 0.5},
            {"income_up_to": float("inf"), "rate": 0.6},],
        "alt_2": [
            {"income_up_to": 24812, "rate": 0.3},
            {"income_up_to": 38098, "rate": 0.4},
            {"income_up_to": 75518, "rate": 0.5},
            {"income_up_to": float("inf"), "rate": 0.6},],}
}


In [8]:
tax_brackets = pd.DataFrame(default_params['tax_brackets'])
print(tax_brackets.columns)

Index(['all', '1', '2', '3', '4', 'alt_1', 'alt_2'], dtype='object')


In [9]:
cutoffs= [0,default_params['tax_brackets']['1'][0]['income_up_to'], default_params['tax_brackets']['1'][1]['income_up_to'], default_params['tax_brackets']['1'][2]['income_up_to'],default_params['tax_brackets']['1'][3]['income_up_to']]
cutoffs

[0, 24812, 38098, 75518, inf]

In [9]:
rate_group1 = [default_params['tax_brackets']['1'][0]['rate'], default_params['tax_brackets']['1'][1]['rate'], default_params['tax_brackets']['1'][2]['rate'], default_params['tax_brackets']['1'][3]['rate']]
rate_group2 = [default_params['tax_brackets']['2'][0]['rate'], default_params['tax_brackets']['2'][1]['rate'], default_params['tax_brackets']['2'][2]['rate'], default_params['tax_brackets']['2'][3]['rate']]
rate_group3 = [default_params['tax_brackets']['3'][0]['rate'], default_params['tax_brackets']['3'][1]['rate'], default_params['tax_brackets']['3'][2]['rate'], default_params['tax_brackets']['3'][3]['rate']]
rate_group4 = [default_params['tax_brackets']['4'][0]['rate'], default_params['tax_brackets']['4'][1]['rate'], default_params['tax_brackets']['4'][2]['rate'], default_params['tax_brackets']['4'][3]['rate']]

In [10]:
tax_rates = pd.DataFrame([rate_group1, rate_group2, rate_group3, rate_group4], columns=['group_1', 'group_2', 'group_3', 'group_4'], index= ['1','2','3','4'])
tax_rates

,group_1,group_2,group_3,group_4
1,0.2,0.4,0.4,0.6
2,0.3,0.3,0.3,0.5
3,0.1,0.1,0.1,0.8
4,0.3,0.4,0.1,0.2


In [11]:
len(data)

100000

In [ ]:
## Model 1

In [70]:
gamma_dict = {}
for i, row in data.iterrows():
    chi = row['income_before_tax']
    for b in [1,2,3,4]:
        lower = cutoffs[b-1]
        upper = cutoffs[b]
        gamma = max(0, min(chi, upper) - lower)
        gamma_dict[(i, b)] = gamma

group_map = data['tax_group'].to_dict()

In [47]:
model = pyo.ConcreteModel()

# Define Sets
model.groups = pyo.Set(initialize=sorted(data["tax_group"].unique()))
model.brackets = pyo.Set(initialize=[int(b) for b in tax_rates.index],ordered=True)
model.taxpayers = pyo.Set(initialize=list(data.index))


# Parameters
model.chi = pyo.Param(model.taxpayers, initialize=data['income_before_tax'].to_dict(), within=pyo.NonNegativeReals)
model.omega = pyo.Param(model.taxpayers, initialize=data['outcome_2'].to_dict(), within=pyo.NonNegativeReals)
model.psi_obs = pyo.Param(model.taxpayers, initialize=(data['income_before_tax']-data['outcome_2']).to_dict(), within=pyo.NonNegativeReals)
model.varphi = pyo.Param(model.brackets, initialize = {b:cutoffs[b] for b in model.brackets}, within=pyo.NonNegativeReals)
model.gamma = pyo.Param(model.taxpayers, model.brackets, initialize=gamma_dict, within=pyo.NonNegativeReals)
model.g_i = pyo.Param(model.taxpayers, initialize=group_map, within=model.groups)


# Decision Variables
model.r = pyo.Var(model.groups, model.brackets, domain=pyo.Reals, bounds=(0,1))
model.s = pyo.Var(model.taxpayers, domain=pyo.NonNegativeReals)


# Objective
@model.Objective(sense=pyo.minimize)
def obj(model):
    return sum(model.s[i] for i in model.taxpayers)

# Constraints:
# -s[1] <= predicted_tax[i] - ψ_obs[i]  <=  s[i]
@model.Constraint(model.taxpayers)
def tax_pressure_lower(m, i):
    g = model.g_i[i]
    expr = sum(m.r[g, b] * m.gamma[i, b] for b in m.brackets) - m.psi_obs[i]
    return expr >= -m.s[i]

@model.Constraint(model.taxpayers)
def tax_pressure_upper(m, i):
    g = model.g_i[i]
    expr = sum(m.r[g, b] * m.gamma[i, b] for b in m.brackets) - m.psi_obs[i]
    return expr <=  m.s[i]

In [48]:
solver = pyo.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

Read LP format model from file /var/folders/np/5830zc454wl5c5qp9x0xxypc0000gn/T/tmpv5gvn_95.pyomo.lp
Reading time = 0.55 seconds
x1: 200000 rows, 100016 columns, 815114 nonzeros
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 24.3.0 24D81)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 200000 rows, 100016 columns and 815114 nonzeros
Model fingerprint: 0x9fa74751
Coefficient statistics:
  Matrix range     [1e+00, 7e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+04]
Presolve time: 0.85s
Presolved: 100016 rows, 200016 columns, 815130 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.10s

Barrier statistics:
 Dense cols : 16
 AA' NZ     : 7.151e+05
 Factor NZ  : 9.152e+05 (roughly 130 MB of memory)
 Factor Ops : 4.486e+06 (less than 1 second per iterati

In [49]:
print("Optimal total slack =", pyo.value(model.obj))

Optimal total slack = 24519.45460683398


In [50]:
import pandas as pd
import matplotlib.pyplot as plt

tax_rates_results = []

# Loop through groups and brackets to extract the optimized tax rates
for g in model.groups:
    for b in model.brackets:
        tax_rates_results.append({
            'Group': g,
            'Bracket': b,
            'Rate': pyo.value(model.r[g, b])
        })

tax_rates_df = pd.DataFrame(tax_rates_results)


In [51]:
# Pivot so that brackets are on x-axis and groups are separate lines
pivot_table = tax_rates_df.pivot(index='Bracket', columns='Group', values='Rate')

In [52]:
tax_rate_table = pivot_table .round(3)
print(tax_rate_table)

Group      1    2    3    4
Bracket                    
1        0.2  0.3  0.1  0.3
2        0.4  0.3  0.1  0.4
3        0.4  0.3  0.1  0.1
4        0.6  0.5  0.8  0.2


In [ ]:
## Model 3

In [ ]:
# Rescale the data

In [12]:
data['income_before_tax'] /= 1000
data['outcome_2'] /= 1000
data['psi_obs'] = data['income_before_tax'] - data['outcome_2']

In [13]:
min_income = data['income_before_tax'].min()
max_income = data['income_before_tax'].max()

print(f"Minimum income: {min_income}")
print(f"Maximum income: {max_income}")


Minimum income: 0.003
Maximum income: 149.998


In [14]:
cutoffs = list(np.linspace(0, 150, 101))  # use same 100 intervals but scaled

In [22]:
print("Income before tax stats:")
print(data["income_before_tax"].describe())

print("\nOutcome_2 stats:")
print(data["outcome_2"].describe())

print("\nObserved tax pressure (psi_obs):")
print((data["income_before_tax"] - data["outcome_2"]).describe())


Income before tax stats:
count    100000.000000
mean         74.964182
std          43.312966
min           0.003000
25%          37.338750
50%          74.982500
75%         112.510250
max         149.998000
Name: income_before_tax, dtype: float64

Outcome_2 stats:
count    100000.000000
mean         51.755077
std          28.018637
min           0.002000
25%          27.767750
50%          55.052000
75%          73.740250
max         118.599000
Name: outcome_2, dtype: float64

Observed tax pressure (psi_obs):
count    100000.000000
mean         23.209105
std          18.307466
min           0.001000
25%           7.169750
50%          18.920000
75%          34.635000
max          69.933000
dtype: float64


In [15]:
gamma_dict = {}
for i, row in data.iterrows():
    chi = row['income_before_tax']
    for b in range(1, 101):
        lower = cutoffs[b-1]
        upper = cutoffs[b]
        gamma = max(0, min(chi, upper) - lower)
        gamma_dict[(i, b)] = gamma

group_map = data['tax_group'].to_dict()

In [24]:
gamma = max(0, min(chi, upper) - lower)
gamma_values = [v for v in gamma_dict.values()]
print(pd.Series(gamma_values).describe())


count    1.000000e+07
mean     7.496418e-01
std      7.474942e-01
min      0.000000e+00
25%      0.000000e+00
50%      7.160000e-01
75%      1.500000e+00
max      1.500000e+00
dtype: float64


In [27]:
model = pyo.ConcreteModel()

# Define Sets
model.groups = pyo.Set(initialize=sorted(data["tax_group"].unique()))
model.candidate_brackets = pyo.Set(initialize=range(1, 101), ordered=True)  # candidate brackets
model.candidate_brackets_no_last = pyo.Set(initialize=range(1, 100))  # for b=1,...,99 only
model.taxpayers = pyo.Set(initialize=list(data.index))

# Parameters
model.chi = pyo.Param(model.taxpayers, initialize=data['income_before_tax'].to_dict(), within=pyo.NonNegativeReals)
model.omega = pyo.Param(model.taxpayers, initialize=data['outcome_2'].to_dict(), within=pyo.NonNegativeReals)
model.psi_obs = pyo.Param(model.taxpayers, initialize=(data['income_before_tax']-data['outcome_2']).to_dict(), within=pyo.NonNegativeReals)
model.varphi = pyo.Param(model.candidate_brackets, initialize={b: cutoffs[b] for b in model.candidate_brackets}, within=pyo.NonNegativeReals)
model.gamma = pyo.Param(model.taxpayers, model.candidate_brackets, initialize=gamma_dict, within=pyo.NonNegativeReals)
model.g_i = pyo.Param(model.taxpayers, initialize=group_map, within=model.groups)

# Decision Variables
model.r = pyo.Var(model.groups, model.candidate_brackets, domain=pyo.Reals, bounds=(0,1))
model.d = pyo.Var(model.candidate_brackets_no_last, domain=pyo.NonNegativeReals)
model.w = pyo.Var(model.candidate_brackets_no_last, domain=pyo.Binary)
model.s = pyo.Var(model.taxpayers, domain=pyo.NonNegativeReals)

# Objective
@model.Objective(sense=pyo.minimize)
def obj(m):
    return sum(m.s[i] for i in m.taxpayers)

# Constraints

# 1. Tax pressure constraints
@model.Constraint(model.taxpayers)
def tax_pressure_lower(m, i):
    g = m.g_i[i]
    expr = sum(m.r[g, b] * m.gamma[i, b] for b in m.candidate_brackets)
    return expr - m.psi_obs[i] >= -m.s[i]

@model.Constraint(model.taxpayers)
def tax_pressure_upper(m, i):
    g = m.g_i[i]
    expr = sum(m.r[g, b] * m.gamma[i, b] for b in m.candidate_brackets)
    return expr - m.psi_obs[i] <= m.s[i]

# 2. Rate increment rule
@model.Constraint(model.groups, model.candidate_brackets_no_last)
def rate_increment(m, g, b):
    return m.r[g, b+1] == m.r[g, b] + m.d[b]

# 3. Increment activation
@model.Constraint(model.candidate_brackets_no_last)
def increment_activation(m, b):
    return m.d[b] <= m.w[b]

# 4. Total number of activated brackets
@model.Constraint()
def total_activated_brackets(m):
    return sum(m.w[b] for b in m.candidate_brackets_no_last) <= 4


In [28]:
solver = pyo.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

Read LP format model from file /var/folders/np/5830zc454wl5c5qp9x0xxypc0000gn/T/tmphj_b_p5x.pyomo.lp
Reading time = 4.17 seconds
x1: 200496 rows, 100598 columns, 10296805 nonzeros
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 24.3.0 24D81)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 200496 rows, 100598 columns and 10296805 nonzeros
Model fingerprint: 0x92753164
Variable types: 100499 continuous, 99 integer (99 binary)
Coefficient statistics:
  Matrix range     [1e-03, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-03, 7e+01]
Found heuristic solution: objective 2320910.4770
Found heuristic solution: objective 2320910.4768
Presolve removed 0 rows and 0 columns (presolve time = 7s)...
Presolve removed 0 rows and 0 columns (presolve time = 10s)...
Presolve removed 0 rows and 0 columns (presolve time = 16s)..

In [29]:
# Objective value
optimal_obj = pyo.value(model.obj)
print(f"Optimal objective value (sum of slack): {optimal_obj:.2f}")


Optimal objective value (sum of slack): 286607.95


In [30]:
selected_brackets = [b for b in model.candidate_brackets_no_last if pyo.value(model.w[b]) > 0.5]
print(f"Selected bracket breakpoints (indices): {selected_brackets}")
print(f"Number of activated brackets: {len(selected_brackets)}")


Selected bracket breakpoints (indices): [15, 18, 56]
Number of activated brackets: 3


In [31]:
selected_cutoffs = [pyo.value(model.varphi[b]) for b in selected_brackets]
print(f"Selected cutoff incomes: {selected_cutoffs}")


Selected cutoff incomes: [np.float64(22.5), np.float64(27.0), np.float64(84.0)]


In [32]:
selected_brackets = [b for b in model.candidate_brackets_no_last if pyo.value(model.w[b]) > 0.5]
selected_brackets = sorted(selected_brackets)

# Extract tax rates for each group at those brackets
selected_rates_by_group = {}

for g in model.groups:
    selected_rates_by_group[g] = {b: round(pyo.value(model.r[g, b]), 4) for b in selected_brackets}


df_rates = pd.DataFrame(selected_rates_by_group).T  # Groups as rows
df_rates.columns = [f"Bracket {b}" for b in df_rates.columns]
df_rates.index.name = "Group"

print(df_rates) 


       Bracket 15  Bracket 18  Bracket 56
Group                                    
1          0.2077      0.2587      0.4298
2          0.1544      0.2054      0.3765
3          0.1001      0.1511      0.3221
4          0.0100      0.0610      0.2321
